In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

%matplotlib inline
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torchsample
from torchsample import transforms as ts_transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import Image
from tensorboard import SummaryWriter
from datetime import datetime
import importlib

from torchsample.transforms import RangeNorm

import functions.fine_tune as ft

plt.ion()   # interactive mode

/home/mtezcan/anaconda3/lib/python3.6/site-packages/torchsample-0.1.2-py3.6.egg/torchsample/datasets.py:16: UserWarning: Cant import nibabel.. Cant load brain images


Load Data
---------



In [2]:
# Data augmentation and normalization for training 
# Just normalization for validation
#uniform_sampler=False
batch_size=32
split=1000
random_seed=1
shuffle=True

data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #ts_transforms.RandomRotate(30)
    ]),
    'val': transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../bin_images/0to5'
dset_train = datasets.ImageFolder(data_dir, data_transforms['train'])
dset_val = datasets.ImageFolder(data_dir, data_transforms['val'])

num_train = len(dset_train)
indices = list(range(num_train))

if shuffle == True:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]
sampler_train = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
sampler_val = torch.utils.data.sampler.SubsetRandomSampler(valid_idx)


dset_loaders = {'train':torch.utils.data.DataLoader(dset_train, batch_size=batch_size,sampler=sampler_train,
                                                    num_workers=12),
                'val':torch.utils.data.DataLoader(dset_val, batch_size=batch_size,sampler=sampler_val,
                                                    num_workers=12)}
'''if(uniform_sampler):
    weights,wpc = ft.make_weights_for_balanced_classes(dsets['train'].imgs, len(dsets['train'].classes))  
    weights = torch.DoubleTensor(weights) 
    sampler = {'train':torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights)) ,
               'val':None}
else:
    sampler = {'train':None,
               'val':None}'''

'''dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}

print(dset_classes)'''

dset_sizes={'train':len(dset_train)-1000,'val':1000}
dset_classes = dset_train.classes
print(dset_classes)
use_gpu = torch.cuda.is_available()

#use_gpu=False
if use_gpu:
    print('GPU is available')
else:
    print('!!!!! NO CUDA GPUS DETECTED')

['0', '1', '2', '3', '4', '5']
GPU is available


In [ ]:
print(len(dset_loaders['train'])*batch_size)

Visualize a few images
^^^^^^^^^^^^^^^^^^^^^^
Let's visualize a few training images so as to understand the data
augmentations.



In [ ]:

inputs, classes = next(iter(dset_loaders['train']))
#print(classes.cpu().numpy().reshape(4,4)+1)
# Make a grid from batch
out = torchvision.utils.make_grid(inputs,nrow=4)
print('Size of the input tensors in one batch after grid is  '+str(out.size()))
plt.figure(figsize=(12,12))
ft.imshow(out, title=[dset_classes[x] for x in classes])

Finetuning the convnet
----------------------

Load a pretrained model and reset final fully connected layer.




In [3]:
def writeLog(logname):
    f=open('runs/'+logname+'/Network_properties.txt','w')
    f.write('Batch size: '+str(batch_size)+'\n')
    f.write('Validation size: '+str(split)+'\n')
    f.write('Random seed: '+str(random_seed)+'\n')
    f.write('Shuffle: '+str(shuffle)+'\n')
    f.write('Validation size: '+str(split)+'\n')
    f.write('Network: '+network+'\n')
    f.write('Criterion: '+criteria+'\n')
    f.write('Learning rate: '+str(lr)+'\n')
    f.write('Momentum: '+str(momentum)+'\n')
    f.write('Leraning Rate Scheduler: '+str(lr_scheduler)+'\n')
    f.write('Leraning Rate Decay Period: '+str(lr_decay_epoch)+'\n')
    f.write('Network is pretrained: '+str(pretrained)+'\n')
    f.write('Network laoded from: '+networkName+'\n')
    f.write('MSE loss function: '+str(mse_loss)+'\n')
    f.close()

In [4]:
import openpyxl
import time


'''iter_loc=12
book = openpyxl.load_workbook('logs.xlsx')
sheet = book.active
sheet.append((1,2,3,4,5,6,7,8,9))
n=sheet.max_row
sheet.cell(row=n,column=7).value=123

book.save('logs.xlsx')'''

def writeLog_xlsx(logname='logs.xlsx',iter_loc=14):
    book = openpyxl.load_workbook(logname)
    sheet = book.active
    if network=='loaded':
        specs=(datetime.now().strftime('%B%d  %H:%M:%S'),networkName,str(split),str(random_seed),str(shuffle),
               optimizer, criteria,str(lr),str(momentum),str(lr_scheduler),str(lr_decay_epoch),str(pretrained),
               str(batch_size))
    else:
        specs=(datetime.now().strftime('%B%d  %H:%M:%S'),network,str(split),str(random_seed),str(shuffle),
               optimizer, criteria,str(lr),str(momentum),str(lr_scheduler),str(lr_decay_epoch),str(pretrained),
               str(batch_size))
    sheet.append(specs)
    current_row = sheet.max_row
    sheet.cell(row=current_row, column=iter_loc+5).value = comment
    #n=sheet.max_row
    #sheet.cell(row=n,column=7).value=123
    book.save(logname)
#writeLog_xlsx()

In [5]:
comment=' ' #'Tested for three rooms'
network='resnet18'
networkName='resnet18_real_sgd_multisoft_August29  19:06:27'
optimizer='sgd'
criteria='multisoft'
iter_loc=14
end_to_end=True
lr=0.01
momentum=0.9
weight_decay=0.0005
lr_scheduler=ft.exp_lr_scheduler
lr_decay_epoch=10
pretrained=True
mse_loss=False
nclasses=6

def network_loader(comment=comment, #'Tested for three rooms'
                    network=network,
                    networkName=networkName,
                    optimizer=optimizer,
                    criteria=criteria,
                    iter_loc=iter_loc,
                    end_to_end=end_to_end,
                    lr=lr,
                    momentum=momentum,
                    weight_decay=weight_decay,
                    lr_scheduler=lr_scheduler,
                    lr_decay_epoch=lr_decay_epoch,
                    pretrained=pretrained,
                    mse_loss=mse_loss,
                    nclasses=nclasses):

    if(criteria=='crossentropy'):
        multilabel=False
    else:
        multilabel=True

    if(network=='resnet18'):
        model_ft = models.resnet18(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet34'):
        model_ft = models.resnet34(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet50'):
        model_ft = models.resnet50(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='resnet101'):
        model_ft = models.resnet101(pretrained=pretrained)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        if(mse_loss):
            model_ft.fc = nn.Linear(num_ftrs, 1)
        else:    
            model_ft.fc = nn.Linear(num_ftrs, nclasses)
    elif(network=='alexnet'):
        model_ft = models.alexnet(pretrained=pretrained)
        num_ftrs = model_ft.classifier[6].out_features
        setattr(model_ft.classifier, '7', nn.ReLU(inplace=True))
        setattr(model_ft.classifier, '8', nn.Dropout())
        setattr(model_ft.classifier, '9', nn.Linear(num_ftrs,nclasses))
    elif(network=='loaded'):
        model_ft = torch.load('./saved_models/'+networkName)
        if not end_to_end:
            for param in model_ft.parameters():
                param.requires_grad = False 
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, nclasses)
    else:
        raise ValueError('Undefined network '+network)

    if use_gpu:
        model_ft = model_ft.cuda()

    if(criteria=='crossentropy'):
        criterion = nn.CrossEntropyLoss()
    elif(criteria=='multisoft'):
        criterion=nn.MultiLabelSoftMarginLoss()
    else:
        raise ValueError('Undefined criteria '+criteria)

    if(optimizer=='adam'):
        optimizer_ft = optim.Adam(model_ft.parameters(),lr=lr,weight_decay=weight_decay)
    elif(optimizer=='sgd'):
        if(end_to_end):
            optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=momentum)#,weight_decay=weight_decay)
        else:
            optimizer_ft = optim.SGD(model_ft.fc.parameters(), lr=lr, momentum=momentum,weight_decay=weight_decay)
    return model_ft, optimizer_ft, criterion, multilabel

In [6]:
importlib.reload(ft)

def run_network():
    model_ft, optimizer_ft, criterion, multilabel = network_loader(comment=comment, #'Tested for three rooms'
                                                                    network=network,
                                                                    networkName=networkName,
                                                                    optimizer=optimizer,
                                                                    criteria=criteria,
                                                                    iter_loc=iter_loc,
                                                                    end_to_end=end_to_end,
                                                                    lr=lr,
                                                                    momentum=momentum,
                                                                    weight_decay=weight_decay,
                                                                    lr_scheduler=lr_scheduler,
                                                                    lr_decay_epoch=lr_decay_epoch,
                                                                    pretrained=pretrained,
                                                                    mse_loss=mse_loss,
                                                                    nclasses=nclasses)
    logname=network+'_'+'_'+optimizer+'_'+criteria+'_'+datetime.now().strftime('%B%d  %H:%M:%S')
    writer = SummaryWriter('runs/'+logname)
    writeLog(logname)
    writeLog_xlsx()
    '''model_ft = ft.train_model(model_ft, criterion, optimizer_ft, lr_scheduler,dset_loaders,dset_sizes,writer,
                            use_gpu=use_gpu,num_epochs=50,batch_size=batch_size,num_log=50,multilabel=multilabel,
                              multi_prob=False,lr_decay_epoch=lr_decay_epoch,init_lr=lr,mse_loss=False,
                              iter_loc=iter_loc)'''

    model_ft = ft.train_model(model_ft, criterion, optimizer_ft, lr_scheduler,dset_loaders,dset_sizes,writer,
                            use_gpu=use_gpu,num_epochs=30,batch_size=batch_size,num_log=100,multilabel=multilabel,
                              multi_prob=False,lr_decay_epoch=lr_decay_epoch,init_lr=lr,mse_loss=mse_loss,
                              iter_loc=iter_loc)
    torch.save(model_ft,'./saved_models/'+logname)
    del model_ft




In [ ]:
mse_loss=False
criteria='crossentropy'
optimizer='adam'
network='resnet18'
momentum=.9
lr=0.01
run_network()

Epoch 0/29
----------
LR is set to 0.01
100/182, acc: 0.2284, CIR-1: 0.6238, RMSE: 1.6095
train Loss: 0.0569 Acc: 0.2491 CIR-1: 0.6680 RMSE 1.5221
val Loss: 0.0515 Acc: 0.2830 CIR-1: 0.7150 RMSE 1.3065

Epoch 1/29
----------
100/182, acc: 0.2628, CIR-1: 0.6963, RMSE: 1.4360


In [ ]:
optimizer='sgd'
network='loaded'
networkName='resnet18_real_sgd_multisoft_August29  19:06:27'
momentum=.9
for learning in [ 0.01, 0.03, 0.05, 0.07, 0.1]:
    lr=learning
    run_network()
    
optimizer='sgd'
network='resnet18'
momentum=.9
for learning in [0.005, 0.01, 0.03, 0.05, 0.07, 0.1]:
    lr=learning
    run_network()
    
criteria='crossentropy'
network='loaded'
networkName='resnet18_real_sgd_multisoft_August29  19:06:27'
momentum=.9
for learning in [0.005, 0.01, 0.03, 0.05, 0.07, 0.1]:
    lr=learning
    run_network()
    
optimizer='sgd'
network='resnet18'
momentum=.9
for learning in [0.005, 0.01, 0.03, 0.05, 0.07, 0.1]:
    lr=learning
    run_network()


In [ ]:
optimizer='sgd'
network='loaded'
networkName='resnet18_real_sgd_multisoft_August29  19:06:27'
momentum=.9
for learning in [0.005, 0.01, 0.03, 0.05, 0.07, 0.1]:
    lr=learning
    run_network()
    
optimizer='sgd'
momentum=.9
for learning in [0.005, 0.01, 0.03, 0.05, 0.07, 0.1]:
    lr=learning
    run_network()
    
    
'''
optimizer='sgd'
network='loaded'
networkName='resnet18_real_sgd_multisoft_August29  19:06:27'
momentum=.9
for learning in [0.005, 0.01, 0.03, 0.05, 0.07, 0.1]:
    lr=learning
    run_network()'''

In [ ]:
optimizer='sgd'
momentum=.9
network='loaded'
networkName='resnet18_real_sgd_multisoft_August29  19:06:27'

for learning in [0.001, 0.005, 0.01, 0.03, 0.05, 0.07, 0.1]:
    lr=learning
    run_network()


In [ ]:
torch.save(model_ft,'./saved_models/'+logname)
model_ft_backup=model_ft

In [ ]:
#Trials
model_ft=torch.load('./saved_models/resnet18_real_sgd_multisoft_August29  19:06:27')
model_ft.train(False)
running_cir1=0
for data in dset_loaders['val']:
    inputs, labels=data
    if use_gpu:
        inputs, labels = Variable(inputs.cuda()), \
            Variable(labels.cuda())
    else:
        inputs, labels = Variable(inputs), Variable(labels)
    outputs = model_ft(inputs)
    #tanh_step=torch.nn.Tanh()
    #outputs=tanh_step(outputs)
    _, preds = torch.max(outputs.data, 1)
    #print(preds)
    running_cir1 += torch.sum(torch.abs(preds - labels.data)<=1)
print(running_cir1/dset_sizes['val'])
print(dset_sizes['val'])

In [ ]:
from PIL import Image
imdir='/home/mtezcan/Documents/test_img/1/JPEG.jpg'
im=im = Image.open(imdir)
im_numpy=data_transforms['val'](im).numpy()
#print(im_numpy.shape)
dsets = datasets.ImageFolder_mtezcan(['//home//mtezcan//Documents//test_img'], data_transforms['val'])

inputs, classes = next(iter(dsets))
inputs=inputs.unsqueeze(0)
inputs=Variable(inputs.cuda())
print(inputs.size())
print(inputs)
outputs = model_ft(inputs)
print(outputs)

In [ ]:
#model_ft=torch.load('./Obsolete/saved_models/resnet18_multi_88_real_7_15_17.mdl')
model_ft=torch.load('./saved_models/'+logname)
if use_gpu:
    model_ft = model_ft.cuda()

#criterion = nn.CrossEntropyLoss()
criterion=nn.MultiLabelSoftMarginLoss()
# Observe that all parameters are being optimized
#optimizer_ft = optim.Adam(model_ft.parameters())
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
model_ft = model_ft.cuda()
print(model_ft)

# --- to-be-optimized ---

In [ ]:
#print(next(iter(dset_loaders['train']))[0])
#model_ft = models.alexnet(pretrained=True)

model_ft=model_ft.cpu()
model_ft.train(False)
#new_classifier = nn.Sequential(*list(model_ft.classifier.children())[:-5])
#model_ft.classifier = new_classifier
#print(model_ft)
model_params= list(model_ft.children())
#model_params[1]=list(model_params[1])
#print(model_params)
new_ft = nn.Sequential(*list(model_params)[:-1])
#print(new_ft)

fvec_tr=np.zeros((20000,512))
label_tr=np.zeros((20000))

fvec_val=np.zeros((20000,512))
label_val=np.zeros((20000))
count=0;

#inputs_t, classes_t = data=next(iter(dset_loaders['train']))
#print(inputs_t.size())
#fvec_t=new_ft(Variable(inputs_t))

for data in dset_loaders['train']:
    inputs_t, classes_t = data
    fvec_t=new_ft(Variable(inputs_t))
    #print(fvec_t)
    fvec_t_cpu=fvec_t.cpu()
    if(fvec_t_cpu.data.numpy().shape[0]==4):
        fvec_tr[count:count+4,:]=fvec_t_cpu.data.numpy().reshape(4,-1)
        label_tr[count:count+4]=classes_t.short().numpy()
        count +=4
fvec_tr=fvec_tr[:count,:]
label_tr=label_tr[:count]


count=0;
for data in dset_loaders['val']:
    inputs_t, classes_t = data
    fvec_t=new_ft(Variable(inputs_t))
    fvec_t_cpu=fvec_t.cpu()
    if(fvec_t_cpu.data.numpy().shape[0]==4):
        fvec_val[count:count+4,:]=fvec_t_cpu.data.numpy().reshape(4,-1)
        label_val[count:count+4]=classes_t.short().numpy()
        count +=4
    
fvec_val=fvec_val[:count,:]
label_val=label_val[:count]

'''
print('The CNN model is')
print(model_ft)
num_ftrs = model_ft.classifier[6].in_features
print('Number of features in the fine tune layer is '+str(num_ftrs))
setattr(model_ft.classifier, '6', nn.Linear(num_ftrs, 9))
#model_ft.classifier['6'] = nn.Linear(num_ftrs, 2)
print(model_ft)
'''

print('Size of the input in training is '+str(fvec_tr.shape))
print('Size of the labels in training is '+str(label_tr.shape))

print('Size of the input in validation is '+str(fvec_val.shape))
print('Size of the labels in validation is '+str(label_val.shape))


In [ ]:
print(fvec_val.shape)
print(fvec_val[1,:])

In [ ]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
#clf = SVC(C=1,kernel='linear',class_weight='balanced',decision_function_shape='ovo')
clf=OneVsOneClassifier(LinearSVC(C=1e-5))
clf.fit(fvec_tr,label_tr)
label_pred=clf.predict(fvec_val)
abs_err=np.abs(label_pred-label_val)
cir1=np.mean(abs_err<=1)
print(cir1)

In [ ]:
from sklearn.svm import SVC

c_val=np.power(2,np.arange(0.,8.,1.))
g_val=np.power(2,np.arange(-10.,-5.,1.))

#c_val=[128]
#g_val=[np.power(2,-10.)]
cir1_max=0.
c_max=0
g_max=0
for c_1 in c_val:
    for g in g_val:
        
        clf_svc = SVC(C=c_1,kernel='rbf',gamma=g,shrinking=False,class_weight='balanced')
        clf=OneVsOneClassifier(clf_svc)
        #print(label_tr)
        clf.fit(fvec_tr,label_tr)

        label_pred=clf.predict(fvec_val)
        #print(label_pred)
        abs_err=np.abs(label_pred-label_val)
        #print(abs_err)
        cir1=np.mean(abs_err<=1)
        print('C = '+str(c_1),', g = ',str(g),', cir-1 = '+str(cir1))
        if(cir1>cir1_max):
            cir1_max=cir1
            c_max=c_1
            g_max=g
            

clf = SVC(C=c_max,kernel='rbf',gamma=g_max)#,class_weight='balanced')
clf.fit(fvec_tr,label_tr)
label_pred=clf.predict(fvec_val)
abs_err=np.abs(label_pred-label_val)
print('Max CIR-1 achieved is '+str(np.mean(abs_err<=1)) +' with c='+str(c_max),', g='+str(g_max))
print('CIR-0 = '+str(np.mean(abs_err<=0)))
print('CIR-2 = '+str(np.mean(abs_err<=2)))
tr_pred=clf.predict(fvec_tr)
abs_tr=np.abs(tr_pred-label_tr)
print('Training CIR-1 is '+str(np.mean(abs_tr<=1)))
#print(abs_err)
#print(label_val)

In [ ]:
model_ft = models.resnet18(pretrained=True)

#print('The CNN model is')
#print(model_ft)
#print(list(model_ft.children())[:-1])
num_ftrs = model_ft.fc.in_features
print('Number of features in the fine tune layer is '+str(num_ftrs))
#setattr(model_ft.classifier, '6', nn.Linear(num_ftrs, 9))
model_ft.fc = nn.Linear(num_ftrs, 9)
#print(model_ft)

#print(next(iter(dset_loaders['train']))[0])
new_ft = nn.Sequential(*list(model_ft.children())[:-1])
#dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=4,
#                                               shuffle=True, num_workers=4)
#                for x in ['train', 'val']}

fvec_tr=np.zeros((X_tr.shape[0],512))
for k in range(0,10,10):#(0,inputs.shape[0],10):
    print(k)
    fvec_now=new_ft(Variable(torch.from_numpy(X_tr[k:k+10,:,:,:])))
    print(fvec_now.numpy())

print(fvec)


'''
print('The CNN model is')
print(model_ft)
num_ftrs = model_ft.classifier[6].in_features
print('Number of features in the fine tune layer is '+str(num_ftrs))
setattr(model_ft.classifier, '6', nn.Linear(num_ftrs, 9))
#model_ft.classifier['6'] = nn.Linear(num_ftrs, 2)
print(model_ft)
'''

if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

Train and evaluate
^^^^^^^^^^^^^^^^^^

It should take around 15-25 min on CPU. On GPU though, it takes less than a
minute.




In [ ]:
visualize_model(model_ft)

ConvNet as fixed feature extractor
----------------------------------

Here, we need to freeze all the network except the final layer. We need
to set ``requires_grad == False`` to freeze the parameters so that the
gradients are not computed in ``backward()``.

You can read more about this in the documentation
`here <http://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward>`__.




In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    #print(param)
    param.requires_grad = False
    
for param in model_conv.fc.parameters():
    #print(param)
    param.requires_grad = True

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 9)

if use_gpu:
    model_conv = model_conv.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

Train and evaluate
^^^^^^^^^^^^^^^^^^

On CPU this will take about half the time compared to previous scenario.
This is expected as gradients don't need to be computed for most of the
network. However, forward does need to be computed.




In [ ]:
model_conv = ft.train_model(model_conv, criterion, optimizer_conv, ft.exp_lr_scheduler,dset_loaders,dset_sizes,writer,
                        use_gpu=use_gpu,num_epochs=100,batch_size=32,num_log=1000,multilabel=False,multi_prob=False,
                         lr_decay_epoch=7)

In [ ]:
visualize_model(model_conv)

plt.ioff()
plt.show()